In [1]:
import os
import pandas as pd
import qiime2 as q2
import requests

from qiime2 import Visualization

data_dir = 'poop_data/Metagenomics'
    
%matplotlib inline

In [2]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [3]:
# path to the picrust2 conda environment - do not change!
picrust_env = '/opt/conda/envs/picrust2/bin'

In [4]:
%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

# append the env location to PATH so that qiime
# can find all required executables
export PATH=$picrust_env:$PATH

$picrust_env/qiime picrust2 full-pipeline \
    --i-seq $'poop_data/Taxonomy'/rep-seqs-filtered_new.qza \
    --i-table $'poop_data/Taxonomy'/table-filtered_new.qza \
    --output-dir $data_dir/picrust2_results \
    --p-placement-tool sepp \
    --p-threads 2 \
    --p-hsp-method pic \
    --p-max-nsti 2 

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


Saved FeatureTable[Frequency] to: poop_data/Metagenomics/picrust2_results/ko_metagenome.qza
Saved FeatureTable[Frequency] to: poop_data/Metagenomics/picrust2_results/ec_metagenome.qza
Saved FeatureTable[Frequency] to: poop_data/Metagenomics/picrust2_results/pathway_abundance.qza


In [3]:
ko = q2.Artifact.load(f'{data_dir}/picrust2_results/ko_metagenome.qza').view(pd.DataFrame)
ec = q2.Artifact.load(f'{data_dir}/picrust2_results/ec_metagenome.qza').view(pd.DataFrame)
pa = q2.Artifact.load(f'{data_dir}/picrust2_results/pathway_abundance.qza').view(pd.DataFrame)

In [4]:
ko.head()

,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K19777,K19778,K19779,K19780,K19784,K19785,K19787,K19788,K19789,K19791
10317.000002929,5142.037624,140.257551,8244.813771,260.071999,6475.569470,0.000155,7614.452333,1306.043795,4242.991858,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,142.764138,0.0,0.0,0.0,318.690457,0.000000
10317.000002930,2563.326126,200.321778,3945.306551,418.262647,2311.037174,12.167427,3720.771736,908.086201,2010.539789,0.000001,...,1.961244e-13,2.599677e-12,1.841890e-15,2.480888e-15,426.041781,0.0,0.0,0.0,248.319698,0.111276
10317.000003283,5190.805498,58.909884,4097.621029,468.584498,4336.160197,6.093604,5776.905030,2098.635538,3172.779917,0.000000,...,1.363630e+03,1.363630e+03,2.720552e+02,2.725722e+02,1590.939135,0.0,0.0,0.0,1642.154746,0.000000
10317.000004079,3883.610863,28.505430,8046.769281,460.999838,7298.148013,79.780128,5404.449520,1809.185532,2847.144885,0.001873,...,2.287358e+01,2.940000e+00,5.865537e-01,5.876683e-01,197.758132,0.0,0.0,0.0,266.304296,0.005843
10317.000016169,3416.638770,164.936054,4919.361142,155.497359,3372.754690,0.012271,4126.667736,1607.982200,3702.155707,0.000000,...,1.762600e+02,1.762600e+02,3.516529e+01,3.523212e+01,351.099496,0.0,0.0,0.0,307.766296,0.000000


In [6]:
metadata = pd.read_csv(f'poop_data/metadata.tsv', sep='\t', header=0, index_col=0)

In [12]:
ko_meta = ko.merge(metadata[['GEN_dog']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['GEN_dog']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['GEN_dog']], left_index=True, right_index=True)

In [33]:
metadata.sort_values('GEN_dog')


,GEN_age_cat,GEN_age_corrected,GEN_bmi_cat,GEN_bmi_corrected,GEN_cat,GEN_collection_timestamp,GEN_country,GEN_dog,GEN_elevation,GEN_geo_loc_name,...,HEA_lung_disease,HEA_mental_illness,HEA_migraine,HEA_seasonal_allergies,HEA_sibo,HEA_skin_condition,HEA_sleep_duration,HEA_smoking_frequency,HEA_thyroid,HEA_weight_change
sampleid,,,,,,,,,,,,,,,,,,,,,
10317.000050,50s,59.0,Overweight,29.98,False,2016-06-27 09:00:00,United Kingdom,False,33.1,United Kingdom:England,...,False,False,False,False,True,False,5-6 hours,Never,False,Increased more than 10 pounds
10317.000053,60s,61.0,Overweight,29.44,False,2016-08-05 10:20:00,United Kingdom,False,163.2,United Kingdom:England,...,True,False,False,False,False,False,7-8 hours,Never,False,Remained stable
10317.000053,40s,47.0,Obese,32.18,False,2016-08-06 08:05:00,United Kingdom,False,172.6,United Kingdom:Wales,...,False,False,False,False,False,False,7-8 hours,Never,False,Remained stable
10317.000027,50s,54.0,Overweight,26.61,False,2016-01-25 09:00:00,USA,False,25.8,USA:VA,...,False,False,False,False,False,False,7-8 hours,Never,False,Decreased more than 10 pounds
10317.000053,60s,64.0,Normal,19.21,False,2016-07-26 15:30:00,United Kingdom,False,79.5,United Kingdom:England,...,False,False,False,False,True,False,7-8 hours,Never,False,Remained stable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000043,Not provided,NaN,Underweight,14.79,False,2016-04-23 10:15:00,United Kingdom,True,6.4,United Kingdom:Unspecified,...,False,False,True,True,False,False,8 or more hours,Not provided,False,Remained stable
10317.000054,60s,61.0,Overweight,28.12,False,2016-07-28 08:20:00,United Kingdom,True,104.3,United Kingdom:England,...,False,True,True,True,False,False,7-8 hours,Never,False,Remained stable
10317.000046,50s,54.0,Normal,22.46,False,2016-07-26 05:00:00,USA,True,83.6,USA:MA,...,True,False,False,True,False,True,6-7 hours,Never,False,Remained stable


In [21]:
# collapse samples per sample_type - calculate average abundance

ko_meta_avg = ko_meta.groupby('GEN_dog').mean()
ec_meta_avg = ec_meta.groupby('GEN_dog').mean()
pa_meta_avg = pa_meta.groupby('GEN_dog').mean()

In [20]:
#why is this empty?? metadata and ko table don't have the same sampleid....

ko_meta_avg.head()

,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K19778,K19779,K19780,K19784,K19785,K19787,K19788,K19789,K19791,GEN_dog


In [15]:
# find top x% of the most abundant KOs, ECs and pathways in each sample type

def find_most_abundant(df: pd.DataFrame, frac):
    if 0 < frac < 1:
        frac = int(frac * len(df.columns))
    print(f'Saving {frac} most abundant features...')
    most_abundant = {
        smp: df.loc[smp, :].sort_values(ascending=False)[:frac]
        for smp in df.index
    }
    return most_abundant

ko_most_abundant = find_most_abundant(ko_meta_avg, 0.01)
ec_most_abundant = find_most_abundant(ec_meta_avg, 0.03)
pa_most_abundant = find_most_abundant(pa_meta_avg, 5)

Saving 101 most abundant features...
Saving 85 most abundant features...
Saving 5 most abundant features...


In [17]:
print(f'10 most abundant KOs in the treatment group are: {ko_most_abundant["True"].index[:10].tolist()}\n'
      f'10 most abundant KOs in the non-treatment group are: {ko_most_abundant["False"].index[:10].tolist()}\n')

KeyError: 'True'

In [18]:
print(f'4 most abundant pathways in the treatment group are: {pa_most_abundant["True"].index[:4].tolist()}\n'
      f'4 most abundant pathways in the non-treatment group are: {pa_most_abundant["False"].index[:4].tolist()}\n')

KeyError: 'True'